In [1]:
import pandas as pd
import numpy as np

import re

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from nltk import word_tokenize

from sklearn.model_selection import train_test_split

from gensim.models import Word2Vec
stopwords = stopwords.words('english')


from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression as log 
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn import metrics    

import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Flatten
from tensorflow.keras.layers import Embedding
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers import Dense, Input, Flatten, Bidirectional
from tensorflow.keras import layers, callbacks

from keras.utils import pad_sequences
from keras.utils.np_utils import to_categorical
import pandas as pd
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm
import numpy as np

In [3]:
df = pd.read_csv('labeled_lyrics_w_genres.csv')
df.head()

df = df.drop(columns = ['Unnamed: 0','Unnamed: 0.1'],axis = 1)

df_dropped = df[(df['genre'] == 'No_genre') | (df['genre'] == 'Non-Music')].index
df.drop(df_dropped, inplace=True, axis='index')

df['genre'].value_counts()

Pop        57357
Rock       26756
Country     7440
Rap         5959
R&B         4773
Name: genre, dtype: int64

In [44]:
df_balanced = ""

cond = df['genre'] == 'Pop'
df_pop = df[cond]
df_pop = df_pop[0:2000]

cond = df['genre'] == 'Rock'
df_rock = df[cond]
df_rock = df_rock[0:2000]
df_rock.shape

cond = df['genre'] == 'Country'
df_country = df[cond]
df_country = df_country[0:2000]
df_country.shape

cond = df['genre'] == 'Rap'
df_rap = df[cond]
df_rap = df_rap[0:2000]
df_rap.shape

cond = df['genre'] == 'R&B'
df_r_b = df[cond]
df_r_b = df_r_b[0:2000]
df_r_b.shape

(2000, 5)

In [13]:
df_balanced = pd.concat([df_pop, df_rock, df_country, df_rap, df_r_b], axis = 0)

df_balanced['genre'].value_counts()

NameError: name 'df_pop' is not defined

In [4]:
#Remove numbers
def remove_numbers(input):
    input = word_tokenize(input)
    without_sw = [word for word in input 
                  if word.isalpha()]
    return ' '.join(without_sw)

# 1. function that makes all text lowercase.
def make_lowercase(test_string):
    return test_string.lower()

# 2. function that removes all punctuation. 
def remove_punc(test_string):
    test_string = re.sub(r'[^\w\s]', '', test_string)
    return test_string

# 3. function that removes all stopwords.
def remove_sw(input):
    input = word_tokenize(input)
    without_sw = [word for word in input 
                  if word not in stopwords]
    return ' '.join(without_sw)

# 4. function to break words into their stem words
def stem_words(input):
    stemming = PorterStemmer()
    tokenized_words = word_tokenize(input)
    
    stemmed_words = [stemming.stem(word) for word in tokenized_words]
    return ' '.join(stemmed_words)

In [5]:
# Pipeline

def text_processing_pipeline(a_string):
    a_string = make_lowercase(a_string)
    a_string = remove_numbers(a_string)
    a_string = remove_punc(a_string)
    a_string = remove_sw(a_string)
    #a_string = stem_words(a_string)
    return a_string

In [6]:
df_balanced = df


In [7]:
df_balanced['seq_clean'] = df_balanced['seq'].apply(lambda x: text_processing_pipeline(x))

In [8]:
df_balanced

,artist,seq,song,label,genre,seq_clean
0,Elijah Blake,"No, no\r\nI ain't ever trapped out the bando\r...",Everyday,0.626,R&B,ai ever trapped bando oh lord get wrong know c...
1,Elijah Blake,"The drinks go down and smoke goes up, I feel m...",Live Till We Die,0.630,Pop,drinks go smoke goes feel got let go cares get...
2,Elijah Blake,She don't live on planet Earth no more\r\nShe ...,The Otherside,0.240,R&B,live planet earth found love venus word said n...
3,Elijah Blake,"Trippin' off that Grigio, mobbin', lights low\...",Pinot,0.536,R&B,trippin grigio mobbin lights low trippin grigi...
4,Elijah Blake,"I see a midnight panther, so gallant and so br...",Shadows & Diamonds,0.371,R&B,see midnight panther gallant brave found found...
...,...,...,...,...,...,...
145245,Michael Franti,Heavy medicine\r\nYa see my eyes are feeling r...,Ganja Babe,0.712,Pop,heavy medicine ya see eyes feeling red bringin...
145246,Rodriguez,The generals hate holidays\r\nOthers shoot up ...,Sandrevan Lullaby - Lifestyles,0.327,Rock,generals hate holidays others shoot chase sun ...
145247,Darryl Worley,Girl tonight it's pouring down on us\r\nIt's b...,Hard Rain Don't Last,0.406,Country,girl tonight pouring us building long yeah dar...
145248,UB40,[Chorus]\r\nThere's a rat in me kitchen what a...,Rat in Mi Kitchen,0.826,Pop,chorus rat kitchen gon na rat kitchen gon na g...


In [10]:
df_balanced.iloc[9][1]

"Each night I lie in my bed \r\nAnd I think about it \r\nOh my dark friend \r\nCan you give me an answer \r\n\r\nThe people that we love\r\nWe cannot understand \r\nEvery day the people that we love are hurt \r\nThe people who we love \r\nAnd cannot understand\r\nWe cannot understand \r\n\r\nDo you believe we're the children of a God\r\nDo you believe \r\nOr do you believe we're left here all alone\r\nDo you believe \r\n\r\nI have this question\r\nWhat is it for \r\nThis poor life \r\nWe live down here\r\n\r\nIs there a power all above \r\nA person so called God \r\nSo called God \r\nWho plans our life \r\nOr is it our free will \r\nThe creator of our life \r\nAre we responsible \r\n\r\nThe people that we love\r\nWe cannot understand \r\nEvery day the people that we love are hurt \r\nThe people who we love \r\nAnd cannot understand\r\nWe cannot understand \r\n\r\nDo you believe we're the children of a God\r\nDo you believe \r\nOr do you believe we're left here all alone\r\nDo you belie

In [9]:
labelencoder = LabelEncoder()
df_balanced['genre'] = labelencoder.fit_transform(df_balanced['genre'])

In [26]:
labelencoder.classes_

array(['Country', 'Pop', 'R&B', 'Rap', 'Rock'], dtype=object)

In [30]:
X = df_balanced['seq_clean']

y = df_balanced['genre'].values
#y = to_categorical(y)

In [34]:
y[145246]

IndexError: index 145246 is out of bounds for axis 0 with size 102285

In [21]:
# Limiting our tokenizers vocab size
max_words = 10000

# create the tokenizer
tokenizer = Tokenizer(num_words=max_words)

# Fit the tokenizer
tokenizer.fit_on_texts(X)

# Create the sequences for each sentence, basically turning each word into its index position
sequences = tokenizer.texts_to_sequences(X)

index_word = tokenizer.index_word

# # Limiting our sequence to only include 500 words
max_length = 300

# # Convert the sequences to all be the same length of 500
X = pad_sequences(sequences, maxlen=max_length, padding='post')

In [22]:
X.shape

(102285, 300)

In [ ]:
# This creates the Neural Network
model = Sequential() 

model.add( Embedding(max_words, 32, input_length=max_length) ) 
#model.add(Flatten())
model.add(Dense(128, activation='relu'))
#model.add(Dense(350, activation='relu'))
#model.add(Dense(400, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 

model.summary()

In [ ]:
model.fit(X,y,
         # validation_data=(X_test,y_test),
          validation_split= 0.2,
          epochs=20,
          batch_size=64)

In [39]:
X.shape

(8500, 300)

In [ ]:
hist = model.fit(X, y, 
                 validation_split=0.2, 
                 epochs=20, batch_size=40)

In [ ]:
# plot the acccuracy

plt.figure(figsize=(8,8))
plt.plot(hist.history['accuracy'], label='training accuracy',color='blue')
plt.plot(hist.history['val_accuracy'], label = 'validation accuracy',color='red', linestyle='dashed')
plt.title('Accuracy by Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
#plt.ylim([0.9, 1.1])
plt.legend(loc='lower right')

## LSTM

In [23]:
#Create LSTM model
model = Sequential()
model.add(layers.Embedding(max_words, 32, input_length=max_length))

model.add(layers.LSTM(64, dropout=0.1))
#model.add(layers.LSTM(74, dropout=0.1))
model.add(layers.Dense(5, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 32)           320000    
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 5)                 325       
                                                                 
Total params: 345,157
Trainable params: 345,157
Non-trainable params: 0
_________________________________________________________________


In [24]:
loss='categorical_crossentropy'
optimizer='adam'
metrics = ['accuracy']

model.compile(loss=loss, optimizer=optimizer, metrics=metrics) 

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

In [26]:
callback = callbacks.EarlyStopping(monitor='val_loss',patience = 2,restore_best_weights=True)
model.fit(X_train,y_train,
          validation_data=(X_test,y_test),
          epochs=20,
          batch_size=64)

Epoch 1/20
1279/1279 [==============================] - 195s 151ms/step - loss: 1.1167 - accuracy: 0.5773 - val_loss: 1.0516 - val_accuracy: 0.5878
Epoch 2/20
1279/1279 [==============================] - 214s 168ms/step - loss: 1.0305 - accuracy: 0.5943 - val_loss: 1.0242 - val_accuracy: 0.5955
Epoch 3/20
1279/1279 [==============================] - 244s 191ms/step - loss: 1.0095 - accuracy: 0.5986 - val_loss: 1.0189 - val_accuracy: 0.5966
Epoch 4/20
1279/1279 [==============================] - 243s 190ms/step - loss: 1.0014 - accuracy: 0.5996 - val_loss: 1.0385 - val_accuracy: 0.5943
Epoch 5/20
1279/1279 [==============================] - 250s 196ms/step - loss: 1.0062 - accuracy: 0.5981 - val_loss: 1.0345 - val_accuracy: 0.5966
Epoch 6/20
1279/1279 [==============================] - 251s 197ms/step - loss: 0.9727 - accuracy: 0.6007 - val_loss: 0.9982 - val_accuracy: 0.5976
Epoch 7/20
1279/1279 [==============================] - 247s 193ms/step - loss: 0.9483 - accuracy: 0.6019 - val_

In [67]:
model.save('LSTM_Genre_model.h5')

In [72]:
from tensorflow import keras
loaded_model = keras.models.load_model('LSTM_Genre_model.h5')

In [74]:
sequences = tokenizer.texts_to_sequences([input])

In [75]:
inputX = pad_sequences(sequences, maxlen=max_length, padding='post')

In [76]:
encode = np.argmax(model.predict(inputX))
encode

1/1 [==============================] - 0s 32ms/step


1

In [73]:
input = '''I never stay in one place too long
A dirt road's singing me a siren song
I gotta find a field
I need to spin my wheels
I got a hankering for four wide tires
And I can't help it it's the way I'm wired
'Fore you get too close
Boy you need to know
I got a heart like a truck
It's been drug through the mud
Runs on dreams and gasoline
And that ole highway holds the key
It's got a lead foot down when it's leaving
Lord knows it's taken a hell of a beating
A little bit of love is all that it's needing
But it's good as it is tough
I got a heart like a truck
There ain't no breaking when I throw it in drive
Don't always keep it in between the lines
If you're ready for a ride pedal down state of mind
Boy I tell you what
You better buckle up'''

input = text_processing_pipeline(input)

In [35]:
sequences = tokenizer.texts_to_sequences([input])

In [36]:
inputX = pad_sequences(sequences, maxlen=max_length, padding='post')

In [23]:
encode = 0

In [24]:
decoded = labelencoder.inverse_transform([encode])
decoded

array(['Country'], dtype=object)

array(['Pop'], dtype=object)

In [60]:
model.fit(X, y, 
          validation_split=0.3, 
          epochs=20, 
          batch_size=20)

Epoch 1/20
350/350 [==============================] - 84s 233ms/step - loss: 1.2026 - accuracy: 0.3594 - val_loss: 4.9718 - val_accuracy: 0.2637
Epoch 2/20
350/350 [==============================] - 84s 239ms/step - loss: 1.1624 - accuracy: 0.3721 - val_loss: 6.1713 - val_accuracy: 0.2053
Epoch 3/20
350/350 [==============================] - 81s 232ms/step - loss: 1.1199 - accuracy: 0.3844 - val_loss: 5.5289 - val_accuracy: 0.1707
Epoch 4/20
350/350 [==============================] - 82s 235ms/step - loss: 1.1180 - accuracy: 0.3917 - val_loss: 5.6625 - val_accuracy: 0.2780
Epoch 5/20
350/350 [==============================] - 80s 230ms/step - loss: 1.0437 - accuracy: 0.4120 - val_loss: 5.7731 - val_accuracy: 0.2827
Epoch 6/20
350/350 [==============================] - 80s 227ms/step - loss: 1.0163 - accuracy: 0.4563 - val_loss: 6.4619 - val_accuracy: 0.2607
Epoch 7/20
350/350 [==============================] - 80s 229ms/step - loss: 0.9655 - accuracy: 0.5154 - val_loss: 6.6143 - val_ac

In [28]:
model.evaluate(X_test,y_test)

7/7 [==============================] - 1s 25ms/step - loss: 2.5711 - accuracy: 0.3650


[2.571089267730713, 0.36500000953674316]

In [70]:
#Create LSTM model
from tensorflow.keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Embedding(max_words, 32, input_length=max_length))

model.add(Bidirectional(layers.LSTM(64, dropout=0.1)))
model.add(layers.Dense(5, activation='softmax'))

model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 300, 32)           800000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              49664     
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 5)                 645       
                                                                 
Total params: 850,309
Trainable params: 850,309
Non-trainable params: 0
_________________________________________________________________


In [72]:
model.fit(X, y, 
          validation_split=0.3, 
          epochs=20, 
          batch_size=20)

Epoch 1/20
350/350 [==============================] - 59s 162ms/step - loss: 1.2516 - accuracy: 0.3513 - val_loss: 5.6959 - val_accuracy: 0.2323
Epoch 2/20
350/350 [==============================] - 56s 161ms/step - loss: 0.9678 - accuracy: 0.5696 - val_loss: 6.0586 - val_accuracy: 0.2297
Epoch 3/20
292/350 [========================>.....] - ETA: 7s - loss: 0.6671 - accuracy: 0.7368

KeyboardInterrupt: 

In [33]:
model.fit(X, y, 
          validation_split=0.3, 
          epochs=20, 
          batch_size=20)

Epoch 1/20
1955/3580 [===============>..............] - ETA: 51:12 - loss: 1.0843 - accuracy: 0.5807

KeyboardInterrupt: 

In [16]:
#Create LSTM model
from tensorflow.keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Embedding(max_words, 32, input_length=max_length))

model.add(layers.LSTM(64, dropout=0.1))
model.add(layers.Dense(5, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 32)           1120000   
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 5)                 325       
                                                                 
Total params: 1,145,157
Trainable params: 1,145,157
Non-trainable params: 0
_________________________________________________________________


In [17]:
loss='categorical_crossentropy'
optimizer='adam'
metrics = ['accuracy']

model.compile(loss=loss, optimizer=optimizer, metrics=metrics) 

In [ ]:
model.fit(X, y, 
          validation_split=0.2, 
          epochs=20, 
          batch_size=20)

Epoch 1/20
4092/4092 [==============================] - 457s 111ms/step - loss: 1.0690 - accuracy: 0.5868 - val_loss: 1.0478 - val_accuracy: 0.5950
Epoch 2/20
4092/4092 [==============================] - 421s 103ms/step - loss: 1.0227 - accuracy: 0.5974 - val_loss: 1.0398 - val_accuracy: 0.5959
Epoch 3/20
4092/4092 [==============================] - 516s 126ms/step - loss: 1.0048 - accuracy: 0.5987 - val_loss: 1.0187 - val_accuracy: 0.5974
Epoch 4/20
4092/4092 [==============================] - 470s 115ms/step - loss: 0.9888 - accuracy: 0.6006 - val_loss: 1.0109 - val_accuracy: 0.5975
Epoch 5/20
4092/4092 [==============================] - 543s 133ms/step - loss: 0.9803 - accuracy: 0.6015 - val_loss: 1.0021 - val_accuracy: 0.5971
Epoch 6/20
4092/4092 [==============================] - 475s 116ms/step - loss: 0.9398 - accuracy: 0.6091 - val_loss: 0.9805 - val_accuracy: 0.6005
Epoch 7/20
4092/4092 [==============================] - 576s 141ms/step - loss: 0.9075 - accuracy: 0.6293 - val_

In [33]:
#Create LSTM model
from tensorflow.keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Embedding(max_words, 32, input_length=max_length))

model.add(layers.LSTM(64, dropout=0.1,  return_sequences=True))
model.add(layers.LSTM(64, dropout=0.1))
model.add(layers.Dense(5, activation='softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 300, 32)           1120000   
                                                                 
 lstm_3 (LSTM)               (None, 300, 64)           24832     
                                                                 
 lstm_4 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 5)                 325       
                                                                 
Total params: 1,178,181
Trainable params: 1,178,181
Non-trainable params: 0
_________________________________________________________________


In [34]:
loss='categorical_crossentropy'
optimizer='adam'
metrics = ['accuracy']

model.compile(loss=loss, optimizer=optimizer, metrics=metrics) 

In [35]:
model.fit(X, y, 
          validation_split=0.2, 
          epochs=20, 
          batch_size=20)

Epoch 1/20
4092/4092 [==============================] - 938s 228ms/step - loss: 1.0581 - accuracy: 0.5894 - val_loss: 1.0476 - val_accuracy: 0.5914
Epoch 2/20
4092/4092 [==============================] - 912s 223ms/step - loss: 1.0223 - accuracy: 0.5981 - val_loss: 1.0404 - val_accuracy: 0.5930
Epoch 3/20
4092/4092 [==============================] - 830s 203ms/step - loss: 0.9995 - accuracy: 0.5996 - val_loss: 1.0077 - val_accuracy: 0.5962
Epoch 4/20
4092/4092 [==============================] - 917s 224ms/step - loss: 0.9555 - accuracy: 0.6018 - val_loss: 0.9938 - val_accuracy: 0.5937
Epoch 5/20
4092/4092 [==============================] - 1128s 276ms/step - loss: 0.9120 - accuracy: 0.6217 - val_loss: 0.9677 - val_accuracy: 0.5967
Epoch 6/20
4092/4092 [==============================] - 954s 233ms/step - loss: 0.8678 - accuracy: 0.6437 - val_loss: 0.9649 - val_accuracy: 0.5919
Epoch 7/20
4092/4092 [==============================] - 926s 226ms/step - loss: 0.8199 - accuracy: 0.6665 - val

In [52]:
#Create LSTM model
from tensorflow.keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Embedding(max_words, 32, input_length=max_length))

model.add(layers.LSTM(64, dropout=0.1,  return_sequences=True))
model.add(Bidirectional(layers.LSTM(64, dropout=0.1)))
model.add(layers.Dense(5, activation='softmax'))

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 400, 32)           1280000   
                                                                 
 lstm_9 (LSTM)               (None, 400, 64)           24832     
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              66048     
 nal)                                                            
                                                                 
 dense_3 (Dense)             (None, 5)                 645       
                                                                 
Total params: 1,371,525
Trainable params: 1,371,525
Non-trainable params: 0
_________________________________________________________________


In [53]:
loss='categorical_crossentropy'
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
metrics = ['accuracy']

model.compile(loss=loss, optimizer= opt, metrics=metrics) 

In [54]:
model.fit(X, y, 
          validation_split=0.2, 
          epochs=20, 
          batch_size=20)

Epoch 1/20
4092/4092 [==============================] - 1535s 374ms/step - loss: 1.0613 - accuracy: 0.5853 - val_loss: 1.0200 - val_accuracy: 0.5943
Epoch 2/20
4092/4092 [==============================] - 1645s 402ms/step - loss: 0.9780 - accuracy: 0.5965 - val_loss: 0.9990 - val_accuracy: 0.5956
Epoch 3/20
4092/4092 [==============================] - 1353s 331ms/step - loss: 0.9362 - accuracy: 0.6054 - val_loss: 0.9868 - val_accuracy: 0.5976
Epoch 4/20
4092/4092 [==============================] - 1391s 340ms/step - loss: 0.9082 - accuracy: 0.6177 - val_loss: 0.9862 - val_accuracy: 0.5930
Epoch 5/20
4092/4092 [==============================] - 4807s 1s/step - loss: 0.8803 - accuracy: 0.6308 - val_loss: 0.9919 - val_accuracy: 0.5900
Epoch 6/20
4092/4092 [==============================] - 1499s 366ms/step - loss: 0.8550 - accuracy: 0.6437 - val_loss: 0.9951 - val_accuracy: 0.5900
Epoch 7/20
4092/4092 [==============================] - 1325s 324ms/step - loss: 0.8329 - accuracy: 0.6551 - 

KeyboardInterrupt: 